# import

In [25]:
import pandas as pd
import numpy as np
from model import GoalProbEvaluator
import yfinance as yf

from utils import get_eom_dates

# All Weather Portfolio

In [26]:
# ALL WEATHER PORTFOLIO (MONTHLY REBALANCED)
df = yf.download(['SPY', 'TLT', 'IEF', 'GLD', 'DBC'])['Adj Close'].dropna()
rtns = np.log(df / df.shift(1)).fillna(0)

rebal_dates = get_eom_dates(rtns, start='2008-1-1', end='2024-6-30')
weights = pd.DataFrame(np.array([[0.3, 0.4, 0.15, 0.075, 0.075]] * len(rebal_dates)), columns=rtns.columns,
                       index=rebal_dates)

[*********************100%***********************]  5 of 5 completed


# Config

- initial_W : Initial capital


- T : Number of rebalancing times


- h : 1 if yearly, 1/4 if quarterly, 1/12 if monthly


- rho : The range of Wealth to be generated
    - The higher the rho value, the lower the probability of generating such Wealth (e.g., 3sigma, 4sigma, ...)
    
    
- bankrupt : The level of Wealth considered to be practical bankruptcy


- imax : The number of Wealth to be generated at each time point (T=t)
    -The more there are, the longer the optimization time
    
    
- lb:  Lower bound for each asset


- ub:  Upper bound for each asset

In [27]:
# INITIALIZE CONFIG
config = {
    'initial_W': 100,
    'T': len(weights.index),
    'bankrupt': 5,  # bankruptcy level
    'h': 1 / 12,  # monthly / if you want to calculate quarterly, set it to 1 / 4
    'imax': 1000,
    'rho': 2.5,   # a scale parameter
    'lb': [0.25, 0.35, 0.10, 0.025, 0.025],  # A little buffer
    'ub': [0.35, 0.45, 0.20, 0.125, 0.125]
}

- er_method: method for estimating expected returns of assets
- cov_method: method for estimating Covariances
- window: window size

In [28]:
est_config = {
    'er_method': 'bayes_stein',
    'cov_method': 'rmt',
    'window': 252 * 30
}

# CashFlows

In [29]:
# Include the scheduled cash flows
cashflows = np.ones((config["T"],)) * 0

# Run

In [30]:
model = GoalProbEvaluator(rtns, weights, config, est_config)
model.add_cashflow(cashflows)

model.fit(fit_dist=True, display_result=True)

********************
WEALTH STATES GENERATED
********************
********************
PATHS GENERATED
********************
Expected Wealth: 100.0 -> 101.1606 -> 102.4545 -> 103.6179 -> 104.6663 -> 105.7394 -> 106.9825 -> 107.9407 -> 108.6719 -> 109.2071 -> 109.136 -> 109.1787 -> 109.3727 -> 109.5756 -> 109.8168 -> 110.111 -> 110.4498 -> 110.9667 -> 111.4911 -> 112.0191 -> 112.6325 -> 113.2574 -> 114.0 -> 114.8832 -> 115.6424 -> 116.3243 -> 117.0264 -> 117.7378 -> 118.5694 -> 119.3648 -> 120.1654 -> 120.9526 -> 121.826 -> 122.8405 -> 123.8838 -> 124.9166 -> 125.9992 -> 127.0321 -> 128.1669 -> 129.3012 -> 130.5697 -> 131.8007 -> 132.9707 -> 134.231 -> 135.6226 -> 136.8005 -> 138.0691 -> 139.2868 -> 140.4072 -> 141.6723 -> 142.9968 -> 144.2012 -> 145.4309 -> 146.5224 -> 147.6028 -> 148.8072 -> 150.0432 -> 151.3257 -> 152.5184 -> 153.739 -> 154.9053 -> 156.093 -> 157.2211 -> 158.3837 -> 159.5037 -> 160.5548 -> 161.4366 -> 162.4194 -> 163.4494 -> 164.422 -> 165.4239 -> 166.3502 -> 167.2731

# Evaluate

- evaluate() arguments
    - goals: scalar if single goal, iterable objects(e.g. tuple, list, etc.) if multi goals
    - init_wealth: wealth value at start_idx (efault value is None, which means config['initial_W'])
    - start_idx: index value of the datetime at which the probability is calculated (default value is 0)
    - end_idx: index value of the datetime at which the goals are expected to achieve ((default value is -1)

## Case 1

- An investor wants to know the probability that the wealth, which was 100(default) in the very beginning, will be greater than 300 at the final datetime

In [32]:
model.evaluate(goals=200)

,start date,end date,wealth(start),wealth(end),Probability
0,2008-01-31,END,100,200,0.7116


- The probability is evaluated to 47.73%

## Case 2

- An investor wants to know the probability that the wealth, which was 200 in January 2020, will be greater than 300 at the final datetime

In [23]:
start_idx = rebal_dates.get_loc(rebal_dates[rebal_dates > '2020-1'][0])
model.evaluate(goals=300, init_wealth=200, start_idx=start_idx, end_idx=-1, decimal=4)

,start date,end date,wealth(start),wealth(end),Probability
0,2020-01-31,END,200,300,0.439


- The probability is evaluated to 41.22%

## Case 3

- An investor wants to know the probability that the wealth, which was 200 in January 2020, will be greater than 220 or 250 in December 2021

In [38]:
start_idx = rebal_dates.get_loc(rebal_dates[rebal_dates > '2020-1'][0])
end_idx = rebal_dates.get_loc(rebal_dates[rebal_dates > '2021-12'][0])
model.evaluate(goals=[220, 250], init_wealth=200, start_idx=start_idx, end_idx=end_idx, decimal=4)

,start date,end date,wealth(start),wealth(end),Probability
0,2020-01-31,2021-12-31,100,120,0.4434
1,2020-01-31,2021-12-31,100,150,0.1866
